In [ ]:
#https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz
#https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Improvement_v1_00.tsv.gz


In [ ]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:14 http://archive.ubuntu.com/ubuntu focal-updates/universe

In [ ]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-01 19:59:58--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2023-03-01 19:59:58 (10.6 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [ ]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"

spark.sparkContext.addFile(url)

toys_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), inferSchema=True, sep='\t', header=True, timestampFormat="mm/dd/yyyy")

# Show DataFrame
toys_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [ ]:
# Get the number of rows in the DataFrame.
toys_df.count()

4864249

# Transform the Data

In [ ]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.

review_toys_df = toys_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_toys_df.show(20)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RDIJS7QYB6XNR|   18778586|B00EDBY7X8|     122952789| 2015-08-31|
|R36ED1U38IELG8|   24769659|B00D7JFOPC|     952062646| 2015-08-31|
| R1UE3RPRGCOLD|   44331596|B002LHA74O|     818126353| 2015-08-31|
|R298788GS6I901|   23310293|B00ARPLCGY|     261944918| 2015-08-31|
|  RNX4EXOBBPN5|   38745832|B00UZOPOFW|     717410439| 2015-08-31|
|R3BPETL222LMIM|   13394189|B009B7F6CA|     873028700| 2015-08-31|
|R3SORMPJZO3F2J|    2749569|B0101EHRSM|     723424342| 2015-08-31|
|R2RDOJQ0WBZCF6|   41137196|B00407S11Y|     383363775| 2015-08-31|
|R2B8VBEPB4YEZ7|     433677|B00FGPU7U2|     780517568| 2015-08-31|
|R1CB783I7B0U52|    1297934|B0013OY0S0|     269360126| 2015-08-31|
| R2D90RQQ3V8LH|   52006292|B00519PJTW|     493486387| 2015-08-31|
|R1Y4ZOUGFMJ327|   32071052|B001TCY2DO|     459122467| 2015-08

## Create the "review_id_table".

In [ ]:
# Get the number of rows in the DataFrame
review_toys_df.count()

4864249

In [ ]:
# Describe the datatypes in the DataFrame
review_toys_df.describe()

DataFrame[summary: string, review_id: string, customer_id: string, product_id: string, product_parent: string, review_date: string]

## Create the "products" Table

In [ ]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = toys_df.select(["product_id", "product_title"])
products_df.show(20)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00EDBY7X8|Monopoly Junior B...|
|B00D7JFOPC|56 Pieces of Wood...|
|B002LHA74O|Super Jumbo Playi...|
|B00ARPLCGY|Barbie Doll and F...|
|B00UZOPOFW|Emazing Lights eL...|
|B009B7F6CA|Melissa & Doug Wa...|
|B0101EHRSM|Big Bang Cosmic P...|
|B00407S11Y|Fun Express Insec...|
|B00FGPU7U2|Fisher-Price Octo...|
|B0013OY0S0|Claw Climber Goli...|
|B00519PJTW|100 Foot Multicol...|
|B001TCY2DO|Pig Jumbo Foil Ba...|
|B00DOQCWF8|Minecraft Animal ...|
|B004C04I4I|Disney Baby: Eeyo...|
|B00NWGEKBY|Team Losi 8IGHT-E...|
|B00000JS5S|Hot Wheels 48- Ca...|
|B00XPWXYDK|ZuZo 2.4GHz 4 CH ...|
|B00VPXX92W|Teenage Mutant Ni...|
|B00YRA3H4U|Franklin Sports M...|
|B009T8BSQY|Alien Frontiers: ...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
print(products_df.count())

4864249


In [ ]:
products_df = products_df.dropDuplicates(["product_id", "product_title"])

In [ ]:
print(products_df.count())

664063


## Create the "customers" Table

In [ ]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = toys_df.groupby("customer_id").agg({"customer_id": "count"})
customers_df.show(20)

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   16989307|                 1|
|   45632184|                 2|
|   14703850|                13|
|   49645387|                 2|
|   16343477|                 1|
|   15554899|                 1|
|   17067926|                 1|
|   50843047|                 2|
|    4051424|                 1|
|   11487525|                 1|
|   19371753|                 1|
|   18634862|                 1|
|   14552054|                 1|
|   52695798|                 1|
|   49438424|                 3|
|   10854449|                 9|
|   48521319|                 1|
|   11839424|                 2|
|   27887950|                 1|
|   45392827|                 3|
+-----------+------------------+
only showing top 20 rows



In [ ]:
from traitlets.config.configurable import Configurable
print(customers_df.count())

2561107


In [ ]:
from pyspark.sql.functions import desc
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count")
customers_df.orderBy(desc("customer_count")).show(20)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   51591392|          1761|
|   49944666|           918|
|   16404530|           858|
|   22788662|           829|
|   52916805|           584|
|    7080939|           583|
|   25968347|           503|
|   15336438|           473|
|   48987445|           413|
|   41694086|           385|
|   45035978|           372|
|   31612235|           356|
|   45450659|           355|
|   10526256|           344|
|   20777467|           332|
|   28845187|           332|
|   43856165|           327|
|   15374732|           327|
|   42700429|           304|
|   50052806|           303|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Get the number of rows in the DataFrame
print(customers_df.count())

2561107


## Create the "vine_table".

In [ ]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = toys_df.select(["review_id", "star_rating", "helpful_votes","total_votes", "vine"])
vine_df.show(20)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RDIJS7QYB6XNR|          5|            0|          0|   N|
|R36ED1U38IELG8|          5|            0|          0|   N|
| R1UE3RPRGCOLD|          2|            1|          1|   N|
|R298788GS6I901|          5|            0|          0|   N|
|  RNX4EXOBBPN5|          1|            1|          1|   N|
|R3BPETL222LMIM|          5|            0|          0|   N|
|R3SORMPJZO3F2J|          3|            2|          2|   N|
|R2RDOJQ0WBZCF6|          5|            0|          0|   N|
|R2B8VBEPB4YEZ7|          5|            0|          0|   N|
|R1CB783I7B0U52|          1|            0|          1|   N|
| R2D90RQQ3V8LH|          5|            0|          0|   N|
|R1Y4ZOUGFMJ327|          5|            0|          0|   N|
|R2BUV9QJI2A00X|          5|            0|          1|   N|
| RSUHRJFJIRB3Z|          4|            

In [ ]:
# Get the number of rows in the DataFrame
vine_df.count()

4864249

# Load

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://mydataclassdb.cy7i019x73of.us-east-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root", "password": "postgres", "driver":"org.postgresql.Driver"}


In [ ]:
# Write review_toys_df to table in RDS
review_toys_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [ ]:

# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table="products_df", mode=mode, properties=config)


In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table="customers_df", mode=mode, properties=config)



In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table="vine_df", mode=mode, properties=config)
